In [33]:
import pandas as pd 
import numpy as np
import geopandas as gpd
import plotly.offline as pyo 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
pd.set_option('display.max_rows', 100)

In [34]:
conceito = pd.read_excel('../dados/Conceito_Enade_Todas_as_IES_2019_XLXS.xlsx')

In [35]:
conceito.head()

,Ano,Código da Área,Área de Avaliação,Código da IES,Nome da IES,Sigla da IES,Organização Acadêmica,Categoria Administrativa,Código do Curso,Modalidade de Ensino,...,Sigla da UF,Nº de Concluintes Inscritos,Nº de Concluintes Participantes,Nota Bruta - FG,Nota Padronizada - FG,Nota Bruta - CE,Nota Padronizada - CE,Conceito Enade (Contínuo),Conceito Enade (Faixa),Observação
0,2019,5,MEDICINA VETERINÁRIA,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT,Universidade,Pública Federal,37,Educação Presencial,...,MT,23,20,42.8150,2.7573,57.8000,3.1363,3.0415,4,NaN
1,2019,5,MEDICINA VETERINÁRIA,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT,Universidade,Pública Federal,100763,Educação Presencial,...,MT,46,39,42.1128,2.6390,55.7282,2.7652,2.7336,3,NaN
2,2019,5,MEDICINA VETERINÁRIA,2,UNIVERSIDADE DE BRASÍLIA,UNB,Universidade,Pública Federal,18031,Educação Presencial,...,DF,129,119,50.6050,4.0697,59.0025,3.3516,3.5312,4,NaN
3,2019,5,MEDICINA VETERINÁRIA,3,UNIVERSIDADE FEDERAL DE SERGIPE,UFS,Universidade,Pública Federal,1108067,Educação Presencial,...,SE,47,39,45.7744,3.2559,56.9667,2.9870,3.0542,4,NaN
4,2019,5,MEDICINA VETERINÁRIA,3,UNIVERSIDADE FEDERAL DE SERGIPE,UFS,Universidade,Pública Federal,1321453,Educação Presencial,...,SE,28,25,43.5400,2.8795,51.7800,2.0580,2.2634,3,NaN


In [36]:
fltr= conceito['Área de Avaliação']== 'ARQUITETURA E URBANISMO'
arq = conceito.loc[fltr]

In [37]:
conceito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8368 entries, 0 to 8367
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Ano                               8368 non-null   int64  
 1   Código da Área                    8368 non-null   int64  
 2   Área de Avaliação                 8368 non-null   object 
 3   Código da IES                     8368 non-null   int64  
 4   Nome da IES                       8368 non-null   object 
 5   Sigla da IES                      7493 non-null   object 
 6   Organização Acadêmica             8368 non-null   object 
 7   Categoria Administrativa          8368 non-null   object 
 8   Código do Curso                   8368 non-null   int64  
 9   Modalidade de Ensino              8368 non-null   object 
 10  Código do Município               8368 non-null   int64  
 11  Município do Curso                8368 non-null   object 
 12  Sigla 

In [7]:
fltr = conceito['Sigla da IES'] == 'UFBA'
conc_UFBA = conceito.loc[fltr]

In [38]:
conceito.groupby(['Sigla da IES'])['Conceito Enade (Faixa)'].count()

Sigla da IES
-                  3
ADJETIVO-CETEP     1
AEMS              17
AFARP              2
AJES               3
                  ..
Uniopet            4
UnirG              7
Univeritas UNG    17
VÉRTICE            6
Área1 Wyden        8
Name: Conceito Enade (Faixa), Length: 959, dtype: int64

In [47]:
arq_ord = arq.sort_values(by=['Conceito Enade (Contínuo)']).reset_index()
print(arq_ord.shape)
ufba_pos = arq_ord['Sigla da IES'] == 'UFBA'
arq_ord.loc[ufba_pos]

(400, 23)


,index,Ano,Código da Área,Área de Avaliação,Código da IES,Nome da IES,Sigla da IES,Organização Acadêmica,Categoria Administrativa,Código do Curso,...,Sigla da UF,Nº de Concluintes Inscritos,Nº de Concluintes Participantes,Nota Bruta - FG,Nota Padronizada - FG,Nota Bruta - CE,Nota Padronizada - CE,Conceito Enade (Contínuo),Conceito Enade (Faixa),Observação
356,1599,2019,21,ARQUITETURA E URBANISMO,578,UNIVERSIDADE FEDERAL DA BAHIA,UFBA,Universidade,Pública Federal,117044,...,BA,61,54,52.3685,3.9361,65.4167,3.5228,3.6262,4,NaN
365,1598,2019,21,ARQUITETURA E URBANISMO,578,UNIVERSIDADE FEDERAL DA BAHIA,UFBA,Universidade,Pública Federal,13266,...,BA,184,147,56.3361,4.5631,65.6735,3.5592,3.8102,4,NaN


In [46]:
arq_ord

,index,Ano,Código da Área,Área de Avaliação,Código da IES,Nome da IES,Sigla da IES,Organização Acadêmica,Categoria Administrativa,Código do Curso,...,Sigla da UF,Nº de Concluintes Inscritos,Nº de Concluintes Participantes,Nota Bruta - FG,Nota Padronizada - FG,Nota Bruta - CE,Nota Padronizada - CE,Conceito Enade (Contínuo),Conceito Enade (Faixa),Observação
0,1784,2019,21,ARQUITETURA E URBANISMO,4030,CENTRO UNIVERSITÁRIO FLUMINENSE,UNIFLU,Centro Universitário,Privada sem fins lucrativos,78154,...,RJ,26,26,31.3269,0.6109,40.5346,0.0000,0.1527,1,NaN
1,1685,2019,21,ARQUITETURA E URBANISMO,1428,INSTITUTO DE ENSINO SUPERIOR PLANALTO,IESPLAN,Faculdade,Privada com fins lucrativos,20723,...,DF,4,4,28.4750,0.1602,43.5500,0.4269,0.3602,1,NaN
2,1734,2019,21,ARQUITETURA E URBANISMO,1965,FACULDADE DE TECNOLOGIA DE ALAGOAS,FAT/AL,Faculdade,Privada sem fins lucrativos,1181199,...,AL,14,14,30.0571,0.4102,43.3000,0.3915,0.3962,1,NaN
3,1455,2019,21,ARQUITETURA E URBANISMO,125,CENTRO UNIVERSITÁRIO DE ARARAS -,UNAR,Centro Universitário,Privada sem fins lucrativos,68190,...,SP,42,22,30.7545,0.5204,43.2136,0.3793,0.4146,1,NaN
4,1801,2019,21,ARQUITETURA E URBANISMO,5544,FACULDADE PRESIDENTE ANTÔNIO CARLOS,FAPAC,Faculdade,Privada com fins lucrativos,72771,...,TO,22,21,33.5286,0.9588,42.3762,0.2607,0.4352,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1590,2019,21,ARQUITETURA E URBANISMO,571,UNIVERSIDADE FEDERAL DO PARANÁ,UFPR,Universidade,Pública Federal,12567,...,PR,66,60,58.8483,4.9601,73.5567,4.6753,4.7465,5,NaN
396,1606,2019,21,ARQUITETURA E URBANISMO,584,UNIVERSIDADE FEDERAL DE GOIÁS,UFG,Universidade,Pública Federal,122584,...,GO,45,38,61.9211,5.0000,73.6289,4.6855,4.7642,5,NaN
397,1722,2019,21,ARQUITETURA E URBANISMO,1810,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",IFSP,"Instituto Federal de Educação, Ciência e Tecno...",Pública Federal,1260216,...,SP,39,36,56.8417,4.6430,75.8500,5.0000,4.9108,5,NaN
398,1553,2019,21,ARQUITETURA E URBANISMO,448,CENTRO UNIVERSITÁRIO RITTER DOS REIS,UNIRITTER,Centro Universitário,Privada com fins lucrativos,1283894,...,RS,1,1,NaN,NaN,NaN,NaN,NaN,SC,NaN


In [106]:
arq_faixa = arq.groupby(['Conceito Enade (Faixa)']).count().reset_index()

arq_faixa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Conceito Enade (Faixa)            6 non-null      object
 1   Ano                               6 non-null      int64 
 2   Código da Área                    6 non-null      int64 
 3   Área de Avaliação                 6 non-null      int64 
 4   Código da IES                     6 non-null      int64 
 5   Nome da IES                       6 non-null      int64 
 6   Sigla da IES                      6 non-null      int64 
 7   Organização Acadêmica             6 non-null      int64 
 8   Categoria Administrativa          6 non-null      int64 
 9   Código do Curso                   6 non-null      int64 
 10  Modalidade de Ensino              6 non-null      int64 
 11  Código do Município               6 non-null      int64 
 12  Município do Curso        

In [222]:
data = [go.Bar(
    x=arq_faixa['Conceito Enade (Faixa)'],
    y=arq_faixa['Ano'],
    marker=dict(color=arq_faixa['Ano'],  colorscale ='tealgrn')
)]
layout = go.Layout(
    width=750, height=400,
    title='Número de cursos de Arquitetura e Urbanismo do Brasil por conceito (Faixa)',
    yaxis=dict(
        title_text="Número de cursos"
    ),
     xaxis=dict(
        title_text="Conceito (Faixa)"
    ),
     hovermode = 'x'
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='BarBrCondceitoFaixa.html')

'BarBrCondceitoFaixa.html'

In [247]:
arq_faixa.sort_values(by='Conceito Enade (Faixa)', inplace =True)

colors = ['chocolate', 'coral', 'cornflowerblue',
                'goldenrod', 'crimson', 'chartreuse']

data = go.Pie( values=arq_faixa['Ano'], labels = arq_faixa['Conceito Enade (Faixa)'], marker=dict( colors=arq_faixa['Ano'])
)

layout={ 'width' : 600, 'height' :500,
        'showlegend': False,
        'title': "Cursos de Arquitetura e Urbanismo do Brasil por conceito (Faixa)"
        }

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='PieBrCondceitoFaixa.html')

'PieBrCondceitoFaixa.html'

In [200]:
uf = pd.read_csv('../dados/brasil.csv', header = 0)
uf.head()

,SIGLA,ESTADO,CAPITAL,REGIAO
0,AC,ACRE,RIO BRANCO,NORTE
1,AM,AMAZONAS,MANAUS,NORTE
2,AP,AMAPÁ,MACAPÁ,NORTE
3,PA,PARÁ,BELÉM,NORTE
4,RO,RONDÔNIA,PORTO VELHO,NORTE


In [202]:
fltr = uf['REGIAO'] == 'NORDESTE'
ne = uf[fltr]

In [206]:
listNe = ne['SIGLA'].unique()

In [216]:
fltr = uf['REGIAO'].isin(['NORDESTE', 'NORTE'])
nne = uf[fltr]

listNNe = nne['SIGLA'].unique()
listNNe

array(['AC', 'AM', 'PA', 'RO', 'RR', 'TO', 'AL', 'BA', 'CE', 'MA', 'PB',
       'PE', 'PI', 'RN', 'SE'], dtype=object)

In [217]:
fltr = arq['Sigla da UF'].isin(listNe)
arq_ne = arq.loc[fltr]
arq_ne.head()

,Ano,Código da Área,Área de Avaliação,Código da IES,Nome da IES,Sigla da IES,Organização Acadêmica,Categoria Administrativa,Código do Curso,Modalidade de Ensino,...,Sigla da UF,Nº de Concluintes Inscritos,Nº de Concluintes Participantes,Nota Bruta - FG,Nota Padronizada - FG,Nota Bruta - CE,Nota Padronizada - CE,Conceito Enade (Contínuo),Conceito Enade (Faixa),Observação
1418,2019,21,ARQUITETURA E URBANISMO,3,UNIVERSIDADE FEDERAL DE SERGIPE,UFS,Universidade,Pública Federal,99444,Educação Presencial,...,SE,57,54,53.2852,4.0810,64.6815,3.4187,3.5843,4,NaN
1420,2019,21,ARQUITETURA E URBANISMO,5,UNIVERSIDADE FEDERAL DO PIAUÍ,UFPI,Universidade,Pública Federal,517,Educação Presencial,...,PI,86,76,51.8487,3.8540,69.6750,4.1257,4.0578,5,NaN
1425,2019,21,ARQUITETURA E URBANISMO,11,UNIVERSIDADE CATÓLICA DE PERNAMBUCO,UNICAP,Universidade,Privada sem fins lucrativos,52971,Educação Presencial,...,PE,101,97,43.4278,2.5232,59.5474,2.6919,2.6497,3,NaN
1519,2019,21,ARQUITETURA E URBANISMO,352,CENTRO UNIVERSITÁRIO DE JOÃO PESSOA,UNIPÊ,Centro Universitário,Privada com fins lucrativos,38551,Educação Presencial,...,PB,225,204,43.2373,2.4931,57.8740,2.4549,2.4645,3,NaN
1526,2019,21,ARQUITETURA E URBANISMO,385,UNIVERSIDADE SALVADOR,UNIFACS,Universidade,Privada com fins lucrativos,20575,Educação Presencial,...,BA,505,440,44.4668,2.6874,56.4891,2.2589,2.3660,3,NaN


In [223]:
arq_faixa_ne = arq_ne.groupby(['Conceito Enade (Faixa)']).count().reset_index()

arq_faixa_ne.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Conceito Enade (Faixa)            5 non-null      object
 1   Ano                               5 non-null      int64 
 2   Código da Área                    5 non-null      int64 
 3   Área de Avaliação                 5 non-null      int64 
 4   Código da IES                     5 non-null      int64 
 5   Nome da IES                       5 non-null      int64 
 6   Sigla da IES                      5 non-null      int64 
 7   Organização Acadêmica             5 non-null      int64 
 8   Categoria Administrativa          5 non-null      int64 
 9   Código do Curso                   5 non-null      int64 
 10  Modalidade de Ensino              5 non-null      int64 
 11  Código do Município               5 non-null      int64 
 12  Município do Curso        

In [225]:
data = [go.Bar(
    x=arq_faixa_ne['Conceito Enade (Faixa)'],
    y=arq_faixa_ne['Ano'],
    marker=dict(color=arq_faixa_ne['Ano'],  colorscale ='peach')
)]
layout = go.Layout(
    width=750, height=400,
    title='Número de cursos de Arquitetura e Urbanismo do Nordeste por conceito (Faixa)',
    yaxis=dict(
        title_text="Número de cursos"
    ),
     xaxis=dict(
        title_text="Conceito (Faixa)"
    ),
     hovermode = 'x'
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='BarNeCondceitoFaixa.html')

'BarNeCondceitoFaixa.html'

In [240]:
arq_faixa_ne  = arq_faixa_ne.sort_values(by='Conceito Enade (Faixa)')

colors = ['chocolate', 'coral', 'cornflowerblue',
                'goldenrod', 'crimson', 'chartreuse']

data = go.Pie( values=arq_faixa_ne['Ano'], labels = arq_faixa_ne['Conceito Enade (Faixa)'], marker=dict( colors=colors)
)

layout={ 'width' : 650, 'height' :600,
        'showlegend': True,
        'title': "Cursos de Arquitetura e Urbanismo do Nodeste por conceito (Faixa)"
        
        }

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='PieNeCondceitoFaixa.html')

'PieNeCondceitoFaixa.html'

In [242]:
fltr = arq['Sigla da UF'].isin(listNNe)
arq_nne = arq.loc[fltr]
arq_faixa_nne = arq_nne.groupby(['Conceito Enade (Faixa)']).count().reset_index()

In [251]:
data = [go.Bar(
    x=arq_faixa_nne['Conceito Enade (Faixa)'],
    y=arq_faixa_nne['Ano'],
    marker=dict(color=arq_faixa_nne['Ano'],  colorscale ='portland')
)]
layout = go.Layout(
    width=800, height=400,
    title='Número de cursos de Arquitetura e Urbanismo do Norte-Nordeste por conceito (Faixa)',
    yaxis=dict(
        title_text="Número de cursos"
    ),
     xaxis=dict(
        title_text="Conceito (Faixa)"
    ),
     hovermode = 'x'
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='BarNNeCondceitoFaixa.html')

'BarNNeCondceitoFaixa.html'

In [245]:
arq_faixa_nne  = arq_faixa_nne.sort_values(by='Conceito Enade (Faixa)')

colors = ['chocolate', 'coral', 'cornflowerblue',
                'goldenrod', 'crimson', 'chartreuse']

data = go.Pie( values=arq_faixa_nne['Ano'], labels = arq_faixa_nne['Conceito Enade (Faixa)'], marker=dict( colors=colors)
)

layout={ 'width' : 650, 'height' :600,
        'showlegend': True,
        'title': "Cursos de Arquitetura e Urbanismo do Nodeste por conceito (Faixa)"
        
        }

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='PieNNeCondceitoFaixa.html')

'PieNNeCondceitoFaixa.html'

In [248]:
fltr = arq['Sigla da UF'] == "BA"
arq_BA = arq.loc[fltr]
arq_faixa_BA = arq_BA.groupby(['Conceito Enade (Faixa)']).count().reset_index()

In [254]:
data = [go.Bar(
    x=arq_faixa_BA['Conceito Enade (Faixa)'],
    y=arq_faixa_BA['Ano'],
    marker=dict(color=arq_faixa_BA['Ano'],  colorscale ='cividis')
)]
layout = go.Layout(
    width=800, height=400,
    title='Número de cursos de Arquitetura e Urbanismo da Bahia por conceito (Faixa)',
    yaxis=dict(
        title_text="Número de cursos"
    ),
     xaxis=dict(
        title_text="Conceito (Faixa)"
    ),
     hovermode = 'x'
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='BarBACondceitoFaixa.html')

'BarBACondceitoFaixa.html'

In [250]:
arq_faixa_BA  = arq_faixa_BA.sort_values(by='Conceito Enade (Faixa)')

colors = ['chocolate', 'coral', 'cornflowerblue',
                'goldenrod', 'crimson', 'chartreuse']

data = go.Pie( values=arq_faixa_BA['Ano'], labels = arq_faixa_BA['Conceito Enade (Faixa)'], marker=dict( colors=colors)
)

layout={ 'width' : 650, 'height' :600,
        'showlegend': True,
        'title': "Cursos de Arquitetura e Urbanismo da Bahia por conceito (Faixa)"
        
        }

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='PieBACondceitoFaixa.html')

'PieBACondceitoFaixa.html'